#  Importing Necessary Packages

In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# 1. Reading the data from csv

In [2]:
data = pd.read_excel(r"C:\Users\Dell\Downloads\Online Retail.xlsx")

# 2. Understand the Data

In [3]:
# Displaying the top 5 rows of the dataset using head()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Display basic information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
# Display statistical summary of the dataset
print(data.describe())

            Quantity                    InvoiceDate      UnitPrice  \
count  541909.000000                         541909  541909.000000   
mean        9.552250  2011-07-04 13:34:57.156386048       4.611114   
min    -80995.000000            2010-12-01 08:26:00  -11062.060000   
25%         1.000000            2011-03-28 11:34:00       1.250000   
50%         3.000000            2011-07-19 17:17:00       2.080000   
75%        10.000000            2011-10-19 11:27:00       4.130000   
max     80995.000000            2011-12-09 12:50:00   38970.000000   
std       218.081158                            NaN      96.759853   

          CustomerID  
count  406829.000000  
mean    15287.690570  
min     12346.000000  
25%     13953.000000  
50%     15152.000000  
75%     16791.000000  
max     18287.000000  
std      1713.600303  


In [6]:
# Displaying number of unique values
data.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

In [7]:
# Checking the datatype of each column
data.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [8]:
# Checking the shape of the dataset that has been loaded
data.shape

(541909, 8)

# 3. Clean the data

In [ ]:
# Convert InvoiceDate to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Fill missing CustomerID values based on InvoiceDate
data['CustomerID'] = data.groupby('InvoiceDate')['CustomerID'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else -1))

# Fill missing Description with a placeholder
data['Description'] = data.groupby('InvoiceNo')['Description'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'No Description'))
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [10]:
data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [11]:
# drop duplicates
data = data.drop_duplicates()

### Handle Outliers in the dataset

In [12]:
# Calculate TotalSpend
data['TotalSpend'] = data['Quantity'] * data['UnitPrice']

# Aggregate data to customer level
customer_data = data.groupby('CustomerID').agg({
    'Quantity': 'sum',
    'TotalSpend': 'sum',
      # Frequency
    'InvoiceNo': 'nunique',
    # For Recency
    'InvoiceDate': 'max'  
}).reset_index()

# Calculate Frequency and Recency
customer_data.rename(columns={'InvoiceNo': 'Frequency'}, inplace=True)
customer_data['Recency'] = (data['InvoiceDate'].max() - customer_data['InvoiceDate']).dt.days

# Drop unnecessary columns
customer_data = customer_data[['CustomerID', 'Quantity', 'TotalSpend', 'Frequency', 'Recency']]

# Inspect the resulting dataframe
customer_data

,CustomerID,Quantity,TotalSpend,Frequency,Recency
0,-1.0,251587,1342813.57,3294,0
1,12346.0,0,0.00,2,325
2,12347.0,2458,4310.00,7,1
3,12348.0,2341,1797.24,4,74
4,12349.0,631,1757.55,1,18
...,...,...,...,...,...
4368,18280.0,45,180.60,1,277
4369,18281.0,54,80.82,1,180
4370,18282.0,98,176.60,3,7
4371,18283.0,1357,2045.53,16,3


In [13]:
def identify_and_remove_outliers(dataset, feature_columns):
    """
    Identifies and removes outliers from the specified columns of the dataset.

    Parameters:
    dataset (DataFrame): The input data.
    feature_columns (list): List of columns to check for outliers.

    Returns:
    DataFrame: The dataset with outliers removed.
    """
    outlier_indices = set()
    
    for feature in feature_columns:
        Q1 = dataset[feature].quantile(0.25)
        Q3 = dataset[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_limit = Q1 - 1.5 * IQR
        upper_limit = Q3 + 1.5 * IQR
        
        # Identify outliers for the current feature
        feature_outliers = dataset[(dataset[feature] < lower_limit) | (dataset[feature] > upper_limit)].index
        outlier_indices.update(feature_outliers)
    
    # Remove identified outliers from the dataset
    cleaned_dataset = dataset.drop(outlier_indices)
    
    return cleaned_dataset

# Apply the function to the customer data
cleaned_customer_data = identify_and_remove_outliers(customer_data, ['Quantity', 'TotalSpend', 'Frequency', 'Recency'])

print(f"Original dataset size: {customer_data.shape[0]}")
print(f"Dataset size after removing outliers: {cleaned_customer_data.shape[0]}")

cleaned_customer_data


Original dataset size: 4373
Dataset size after removing outliers: 3663


,CustomerID,Quantity,TotalSpend,Frequency,Recency
1,12346.0,0,0.00,2,325
4,12349.0,631,1757.55,1,18
5,12350.0,197,334.40,1,309
6,12352.0,470,1545.41,11,35
7,12353.0,20,89.00,1,203
...,...,...,...,...,...
4367,18278.0,66,173.90,1,73
4368,18280.0,45,180.60,1,277
4369,18281.0,54,80.82,1,180
4370,18282.0,98,176.60,3,7


#  4. Transform the Data

In [16]:
# Normalize the data
scaler = StandardScaler()
scaled_columns = scaler.fit_transform(cleaned_customer_data[['Quantity', 'TotalSpend', 'Frequency', 'Recency']])

# Convert the scaled features back to a DataFrame
scaled_customer_data = pd.DataFrame(scaled_columns, columns=['Quantity', 'TotalSpend', 'Frequency', 'Recency'])
scaled_customer_data

,Quantity,TotalSpend,Frequency,Recency
0,-1.033639,-1.082133,-0.477305,2.586083
1,0.317000,1.233102,-0.860718,-0.798631
2,-0.611966,-0.641625,-0.860718,2.409681
3,-0.027616,0.953648,2.973419,-0.611204
4,-0.990830,-0.964893,-0.860718,1.241018
...,...,...,...,...
3658,-0.892368,-0.853053,-0.860718,-0.192249
3659,-0.937318,-0.844227,-0.860718,2.056877
3660,-0.918054,-0.975668,-0.860718,0.987440
3661,-0.823873,-0.849496,-0.093891,-0.919907
